In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.utils import shuffle

# Define seed
SEED = 42

# Set the seed for reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU setups
torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
torch.backends.cudnn.benchmark = False    # Disable if you want deterministic results

# Mapear todos os ataques
map_attacks = {'Benign':0, 
              'DoS':1,
              'dos':1,
              'Reconnaissance':2,
              'DDoS':3,
              'ddos':3,
              'Theft':4,
              'ransomware':5,
              'scanning':6,
              'xss':7,
              'mitm':8,
              'password':9,
              'Backdoor':10,
              'backdoor':10,
              'Analysis':11,
              'Generic':12,
              'Exploits':13,
              'Bot':14,
              'DoS_attacks-SlowHTTPTest':15,
              'SSH-Bruteforce':16,
              'DoS_attacks-Hulk':17,
              'Infilteration':18,
              'DDOS_attack-HOIC':19, 
              'FTP-BruteForce':20,
              'Brute_Force_-Web':21, 
              'DDoS_attacks-LOIC-HTTP':22, 
              'DoS_attacks-GoldenEye':23,
              'DDOS_attack-LOIC-UDP':24, 
              'DoS_attacks-Slowloris':25, 
              'Brute_Force_-XSS':26,
              'SQL_Injection':27
              }

all_preds_lists = []
all_labels_lists = []
all_attacks_lists = []

In [2]:
def get_datasets(dataset):

    atributos_comuns = [
                #'L4_SRC_PORT',
                #'L4_DST_PORT',
                'PROTOCOL',
                #'L7_PROTO',
                'IN_BYTES',
                'IN_PKTS',
                'OUT_BYTES',
                'OUT_PKTS',
                #'TCP_FLAGS',
                #'CLIENT_TCP_FLAGS',
                #'SERVER_TCP_FLAGS',
                'FLOW_DURATION_MILLISECONDS',
                'DURATION_IN', 
                'DURATION_OUT',
                #'MIN_TTL', 
                #'MAX_TTL',
                'LONGEST_FLOW_PKT',
                #'SHORTEST_FLOW_PKT',
                #'MIN_IP_PKT_LEN', 
                'MAX_IP_PKT_LEN', 
                'SRC_TO_DST_SECOND_BYTES', 
                'DST_TO_SRC_SECOND_BYTES',
                'RETRANSMITTED_IN_BYTES', 
                'RETRANSMITTED_IN_PKTS', 
                'RETRANSMITTED_OUT_BYTES', 
                'RETRANSMITTED_OUT_PKTS',
                'SRC_TO_DST_AVG_THROUGHPUT', 
                'DST_TO_SRC_AVG_THROUGHPUT', 
                'NUM_PKTS_UP_TO_128_BYTES',
                'NUM_PKTS_128_TO_256_BYTES', 
                'NUM_PKTS_256_TO_512_BYTES', 
                'NUM_PKTS_512_TO_1024_BYTES',
                'NUM_PKTS_1024_TO_1514_BYTES', 
                #'TCP_WIN_MAX_IN', 
                #'TCP_WIN_MAX_OUT', 
                'ICMP_TYPE', 
                'ICMP_IPV4_TYPE',
                #'DNS_QUERY_ID', 
                'DNS_QUERY_TYPE', 
                #'DNS_TTL_ANSWER',
                #'FTP_COMMAND_RET_CODE',
                'Label', 
                'Attack']
    
    if dataset == "dataset1":
         #df = pd.read_csv('datasets/BoT-IoT.csv') # NF-BoT-IoT-v2
         df = pd.read_csv('datasets/dataset1v2.csv')
    if dataset == "dataset2":
         #df = pd.read_csv('datasets/ToN-IoT.csv') # NF-ToN-IoT-v2
         df = pd.read_csv('datasets/dataset2v2.csv')
    if dataset == "dataset3":
         #df = pd.read_csv('datasets/UNSW-NB15.csv') # NF-UNSW-NB15-v2
         df = pd.read_csv('datasets/dataset3v2.csv')
    if dataset == "dataset4":
         #df = pd.read_csv('datasets/CICIDS2018.csv') # NF-CICIDS2018-v3
         df = pd.read_csv('datasets/dataset4v2.csv')

    dataframe = df[atributos_comuns]
        
    return dataframe

In [3]:
def preprocessing_data(df):
    df_source = df
    df_source_label = df_source['Attack']
    df_source = df_source.drop('Attack',axis=1)
    df_source = df_source.drop('Label',axis=1)
    
    # normalize
    scaler = MinMaxScaler()
    df_source_scaled = scaler.fit_transform(df_source)
    df_source = pd.DataFrame(df_source_scaled, columns=df_source.columns)
        
    df_source_label = df_source_label.to_frame()
    x_source1_train, x_source1_test, y_source1_train, y_source1_test = train_test_split(df_source, df_source_label, test_size = 0.30)
    
    #y_source1_train['Attack'] = y_source1_train['Attack'].apply(lambda x: 0 if x == 'Benign' else 1)
    y_source1_train['Attack']=y_source1_train['Attack'].map(map_attacks)
    z_split_test = y_source1_test.copy()
    #y_source1_test['Attack'] = y_source1_test['Attack'].apply(lambda x: 0 if x == 'Benign' else 1)
    y_source1_test['Attack']=y_source1_test['Attack'].map(map_attacks)
    z_split_test['Attack']=z_split_test['Attack'].map(map_attacks)
    
    return df_source, df_source_label, x_source1_train, x_source1_test, y_source1_train, y_source1_test, z_split_test

In [4]:
def inicia_datasets(treino):
      df = get_datasets(treino)
      print(f"Prepara {treino}: {df.shape}")
      df, y_label, x_train, x_test, y_train, y_test, z_test = preprocessing_data(df)
      print( x_train.shape, y_train.shape)
      print( x_test.shape, y_test.shape, z_test.shape)
      return df, y_label, x_train, x_test, y_train, y_test, z_test

In [5]:
treino = "dataset1" 
df1, y1_label, x1_train, x1_test, y1_train, y1_test, z1_test = inicia_datasets(treino)

Prepara dataset1: (224511, 28)
(157157, 26) (157157, 1)
(67354, 26) (67354, 1) (67354, 1)


In [6]:
treino = "dataset2" 
df2, y2_label, x2_train, x2_test, y2_train, y2_test, z2_test = inicia_datasets(treino)

Prepara dataset2: (224512, 28)
(157158, 26) (157158, 1)
(67354, 26) (67354, 1) (67354, 1)


In [7]:
treino = "dataset3" 
df3, y3_label, x3_train, x3_test, y3_train, y3_test, z3_test = inicia_datasets(treino)

Prepara dataset3: (142152, 28)
(99506, 26) (99506, 1)
(42646, 26) (42646, 1) (42646, 1)


In [8]:
treino = "dataset4" 
df4, y4_label, x4_train, x4_test, y4_train, y4_test, z4_test = inicia_datasets(treino)

Prepara dataset4: (223976, 28)
(156783, 26) (156783, 1)
(67193, 26) (67193, 1) (67193, 1)


In [9]:
# --- Concatena os dados de treinamento ---
x_train_concat = pd.concat([x1_train, x2_train, x3_train, x4_train], axis=0)
y_train_concat = pd.concat([y1_train, y2_train, y3_train, y4_train], axis=0)

# --- Embaralha os dados concatenados ---
x_train_concat, y_train_concat = shuffle(x_train_concat, y_train_concat, random_state=42)

# Seleciono Classificador
clf=RandomForestClassifier(n_estimators=100, random_state=42)

# Treinamento
clf.fit(x_train_concat, y_train_concat)

/usr/lib/python3/dist-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [10]:
# Predição no dataset 1
y1_pred = clf.predict(x1_test)

# Métricas
acc_1 = accuracy_score(y1_test, y1_pred)
f1_1 = f1_score(y1_test, y1_pred, average='weighted')
cm_1 = confusion_matrix(y1_test, y1_pred)

# Mapeando as classes reais para os labels de ataque
mapped_labels = [list(map_attacks.keys())[list(map_attacks.values()).index(label)] for label in np.unique(y1_test)]

# Calculando o F1-Score por classe (por ataque)
f1_per_class = f1_score(y1_test, y1_pred, average=None)  # F1 por classe

# Calculando TPR e TNR para cada classe no caso de multi-classe
for i, label in enumerate(np.unique(y1_test)):
    # Encontrando os índices da classe no y_test e y_pred
    TP = cm_1[i, i]  # Verdadeiros positivos para a classe i
    FN = np.sum(cm_1[i, :]) - TP  # Falsos negativos para a classe i
    FP = np.sum(cm_1[:, i]) - TP  # Falsos positivos para a classe i
    TN = np.sum(cm_1) - (TP + FN + FP)  # Verdadeiros negativos para a classe i

    # Cálculo do TPR (recall)
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    # Cálculo do TNR (especificidade)
    TNR = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    # Imprimindo o F1-Score por classe
    print(f'Classe {mapped_labels[i]} - F1-Score: {f1_per_class[i]:.4f}, TPR: {TPR:.4f}, TNR: {TNR:.4f}')

# Imprimindo as métricas gerais
print("Acurácia: ", acc_1)
print("F1 Score (Weighted): ", f1_1)

# Armazenando as previsões e rótulos para análise posterior
all_preds_lists.extend(y1_pred)
all_labels_lists.extend(y1_test)
all_attacks_lists.extend(z1_test)


Classe Benign - F1-Score: 0.9773, TPR: 0.9815, TNR: 0.9729
Classe DoS - F1-Score: 0.9830, TPR: 0.9812, TNR: 0.9971
Classe Reconnaissance - F1-Score: 0.9253, TPR: 0.9239, TNR: 0.9857
Classe DDoS - F1-Score: 0.9863, TPR: 0.9759, TNR: 0.9994
Classe Theft - F1-Score: 0.9770, TPR: 0.9877, TNR: 0.9996
Acurácia:  0.9712118062772812
F1 Score (Weighted):  0.9712162948272384


In [11]:
# Predição no dataset 2
y2_pred = clf.predict(x2_test)

# Métricas
acc_2 = accuracy_score(y2_test, y2_pred)
f1_2 = f1_score(y2_test, y2_pred, average='weighted')
cm_2 = confusion_matrix(y2_test, y2_pred)

# Mapeando as classes reais para os labels de ataque
mapped_labels = [list(map_attacks.keys())[list(map_attacks.values()).index(label)] for label in np.unique(y2_test)]

# Calculando o F1-Score por classe (por ataque)
f1_per_class = f1_score(y2_test, y2_pred, average=None)  # F1 por classe

# Calculando TPR e TNR para cada classe no caso de multi-classe
for i, label in enumerate(np.unique(y2_test)):
    # Encontrando os índices da classe no y_test e y_pred
    TP = cm_2[i, i]  # Verdadeiros positivos para a classe i
    FN = np.sum(cm_2[i, :]) - TP  # Falsos negativos para a classe i
    FP = np.sum(cm_2[:, i]) - TP  # Falsos positivos para a classe i
    TN = np.sum(cm_2) - (TP + FN + FP)  # Verdadeiros negativos para a classe i

    # Cálculo do TPR (recall)
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    # Cálculo do TNR (especificidade)
    TNR = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    # Imprimindo o F1-Score por classe
    print(f'Classe {mapped_labels[i]} - F1-Score: {f1_per_class[i]:.4f}, TPR: {TPR:.4f}, TNR: {TNR:.4f}')

# Imprimindo as métricas gerais
print("Acurácia: ", acc_2)
print("F1 Score (Weighted): ", f1_2)

# Armazenando as previsões e rótulos para análise posterior
all_preds_lists.extend(y2_pred)
all_labels_lists.extend(y2_test)
all_attacks_lists.extend(z2_test)


Classe Benign - F1-Score: 0.9767, TPR: 0.9841, TNR: 0.9689
Classe DoS - F1-Score: 0.8220, TPR: 0.9109, TNR: 0.9791
Classe DDoS - F1-Score: 0.9734, TPR: 0.9668, TNR: 0.9971
Classe ransomware - F1-Score: 0.9822, TPR: 0.9704, TNR: 0.9999
Classe scanning - F1-Score: 0.8831, TPR: 0.8328, TNR: 0.9963
Classe xss - F1-Score: 0.9116, TPR: 0.9537, TNR: 0.9904
Classe mitm - F1-Score: 0.6017, TPR: 0.4513, TNR: 0.9982
Classe password - F1-Score: 0.9459, TPR: 0.9425, TNR: 0.9965
Classe Backdoor - F1-Score: 0.9980, TPR: 0.9986, TNR: 0.9998
Acurácia:  0.9446951925646584
F1 Score (Weighted):  0.9423189444293827


In [12]:
# Predição no dataset 3
y3_pred = clf.predict(x3_test)

# Métricas
acc_3 = accuracy_score(y3_test, y3_pred)
f1_3 = f1_score(y3_test, y3_pred, average='weighted')
cm_3 = confusion_matrix(y3_test, y3_pred)

# Mapeando as classes reais para os labels de ataque
mapped_labels = [list(map_attacks.keys())[list(map_attacks.values()).index(label)] for label in np.unique(y3_test)]

# Calculando o F1-Score por classe (por ataque)
f1_per_class = f1_score(y3_test, y3_pred, average=None)  # F1 por classe

# Calculando TPR e TNR para cada classe no caso de multi-classe
for i, label in enumerate(np.unique(y3_test)):
    # Encontrando os índices da classe no y_test e y_pred
    TP = cm_3[i, i]  # Verdadeiros positivos para a classe i
    FN = np.sum(cm_3[i, :]) - TP  # Falsos negativos para a classe i
    FP = np.sum(cm_3[:, i]) - TP  # Falsos positivos para a classe i
    TN = np.sum(cm_3) - (TP + FN + FP)  # Verdadeiros negativos para a classe i

    # Cálculo do TPR (recall)
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    # Cálculo do TNR (especificidade)
    TNR = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    # Imprimindo o F1-Score por classe
    print(f'Classe {mapped_labels[i]} - F1-Score: {f1_per_class[i]:.4f}, TPR: {TPR:.4f}, TNR: {TNR:.4f}')

# Imprimindo as métricas gerais
print("Acurácia: ", acc_3)
print("F1 Score (Weighted): ", f1_3)

# Armazenando as previsões e rótulos para análise posterior
all_preds_lists.extend(y3_pred)
all_labels_lists.extend(y3_test)
all_attacks_lists.extend(z3_test)


Classe Benign - F1-Score: 0.9961, TPR: 0.9942, TNR: 0.9980
Classe DoS - F1-Score: 0.3255, TPR: 0.3074, TNR: 0.9749
Classe Reconnaissance - F1-Score: 0.7889, TPR: 0.7667, TNR: 0.9829
Classe Backdoor - F1-Score: 0.1414, TPR: 0.1260, TNR: 0.9903
Classe Analysis - F1-Score: 0.1246, TPR: 0.1411, TNR: 0.9808
Classe Generic - F1-Score: 0.8704, TPR: 0.8640, TNR: 0.9839
Classe Exploits - F1-Score: 0.8424, TPR: 0.8650, TNR: 0.9463
Acurácia:  0.8749003423533274
F1 Score (Weighted):  0.8742293104358407


In [13]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

# Predição no dataset 4
y4_pred = clf.predict(x4_test)

# Métricas
acc_4 = accuracy_score(y4_test, y4_pred)
f1_4 = f1_score(y4_test, y4_pred, average='weighted')
cm_4 = confusion_matrix(y4_test, y4_pred)

# Mapeando as classes reais para os labels de ataque
mapped_labels = [list(map_attacks.keys())[list(map_attacks.values()).index(label)] for label in np.unique(y4_test)]

# Calculando o F1-Score por classe (por ataque)
f1_per_class = f1_score(y4_test, y4_pred, average=None)  # F1 por classe

# Calculando TPR e TNR para cada classe no caso de multi-classe
for i, label in enumerate(np.unique(y4_test)):
    # Encontrando os índices da classe no y_test e y_pred
    TP = cm_4[i, i]  # Verdadeiros positivos para a classe i
    FN = np.sum(cm_4[i, :]) - TP  # Falsos negativos para a classe i
    FP = np.sum(cm_4[:, i]) - TP  # Falsos positivos para a classe i
    TN = np.sum(cm_4) - (TP + FN + FP)  # Verdadeiros negativos para a classe i

    # Cálculo do TPR (recall)
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    # Cálculo do TNR (especificidade)
    TNR = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    # Imprimindo o F1-Score por classe
    print(f'Classe {mapped_labels[i]} - F1-Score: {f1_per_class[i]:.4f}, TPR: {TPR:.4f}, TNR: {TNR:.4f}')

# Imprimindo as métricas gerais
print("Acurácia: ", acc_4)
print("F1 Score (Weighted): ", f1_4)

# Armazenando as previsões e rótulos para análise posterior
all_preds_lists.extend(y4_pred)
all_labels_lists.extend(y4_test)
all_attacks_lists.extend(z4_test)


Classe Benign - F1-Score: 0.9617, TPR: 0.9910, TNR: 0.9303
Classe Bot - F1-Score: 1.0000, TPR: 1.0000, TNR: 1.0000
Classe DoS_attacks-SlowHTTPTest - F1-Score: 0.0000, TPR: 0.0000, TNR: 1.0000
Classe SSH-Bruteforce - F1-Score: 1.0000, TPR: 1.0000, TNR: 1.0000
Classe DoS_attacks-Hulk - F1-Score: 0.9997, TPR: 0.9997, TNR: 1.0000
Classe Infilteration - F1-Score: 0.4759, TPR: 0.3351, TNR: 0.9962
Classe DDOS_attack-HOIC - F1-Score: 1.0000, TPR: 1.0000, TNR: 1.0000
Classe FTP-BruteForce - F1-Score: 0.6927, TPR: 1.0000, TNR: 0.9531
Classe Brute_Force_-Web - F1-Score: 0.7422, TPR: 0.6829, TNR: 0.9989
Classe DDoS_attacks-LOIC-HTTP - F1-Score: 0.9830, TPR: 0.9669, TNR: 1.0000
Classe DoS_attacks-GoldenEye - F1-Score: 0.9980, TPR: 0.9997, TNR: 0.9998
Classe DDOS_attack-LOIC-UDP - F1-Score: 0.9966, TPR: 1.0000, TNR: 0.9999
Classe DoS_attacks-Slowloris - F1-Score: 0.9983, TPR: 0.9980, TNR: 0.9999
Classe Brute_Force_-XSS - F1-Score: 0.5385, TPR: 0.8630, TNR: 0.9971
Classe SQL_Injection - F1-Score: 0.1

In [14]:
from sklearn.metrics import f1_score
import pandas as pd

# Supondo que você já tenha os dados de todas as predições e labels
all_preds = all_preds_lists  # Supondo que você já tenha as predições
all_labels = pd.concat([y1_test, y2_test, y3_test, y4_test], axis=0).values.tolist()
all_attacks = pd.concat([y1_test, y2_test, y3_test, y4_test], axis=0).values.tolist()

# Inverter dicionário para mapear ataques de forma invertida
map_attacks_inv = {v: k for k, v in map_attacks.items()}

# Criar o DataFrame com as predições, labels e ataques
df = pd.DataFrame({
    "Real": all_labels,
    "Previsto": all_preds,
    "Ataque": all_attacks
})

# Certificar-se de que as colunas "Real" e "Ataque" são listas de valores
df['Real'] = df['Real'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['Ataque'] = df['Ataque'].apply(lambda x: x[0] if isinstance(x, list) else x)

# Criar coluna de acerto/erro
df["Acertou"] = df["Real"] == df["Previsto"]

# Agrupar predições por tipo de ataque
tabela = df.groupby("Ataque").agg(
    Total=("Real", "count"),
    Acertos=("Acertou", "sum")
)

# Adicionar Acurácia por ataque
tabela["Acurácia"] = tabela["Acertos"] / tabela["Total"]

# Mapear para nomes de ataques
tabela.index = tabela.index.map(map_attacks_inv)

# Ordenar a tabela pela maior acurácia (decrescente)
tabela = tabela.sort_values(by="Acurácia", ascending=False)

# Exibir o resumo final
print("Resumo por tipo de ataque:")
print(tabela)

# Calcular o F1 Score Global
f1 = f1_score(all_labels, all_preds, average='macro')  # F1 Score macro, para considerar todas as classes igualmente

print(f"F1 Score Macro: {f1:.4f}")


Resumo por tipo de ataque:
                           Total  Acertos  Acurácia
Ataque                                             
FTP-BruteForce              3373     3373  1.000000
DDOS_attack-HOIC            3346     3346  1.000000
SSH-Bruteforce              3232     3232  1.000000
Bot                         3303     3303  1.000000
DDOS_attack-LOIC-UDP        1033     1033  1.000000
DoS_attacks-GoldenEye       3005     3004  0.999667
DoS_attacks-Hulk            2976     2975  0.999664
DoS_attacks-Slowloris       3027     3021  0.998018
Theft                        732      723  0.987705
Benign                    122262   120679  0.987052
ddos                       19576    19026  0.971904
ransomware                  1049     1018  0.970448
DDoS_attacks-LOIC-HTTP      3296     3187  0.966930
xss                         4359     4157  0.953659
password                    4329     4080  0.942481
dos                        17018    15207  0.893583
backdoor                    4875     